In [1]:
from src.models.linreg import LinReg

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm

In [2]:
np.random.seed(69)
x = np.linspace(0, 5, 50)
z = np.linspace(3, 8, 50)
y = 1 + 3 * x + 5*z + np.random.normal(0, 2, 50)
data = pd.DataFrame({'outcome': y,
                     'x': x,
                     'z': z})

In [3]:
linreg = LinReg(df=data, outcome="outcome", independent=["z", "x^4"])
linreg.independent_vars

['z', 'x^4']


['x^4', 'z', 'x^3', 'x^2']

In [4]:
linreg.data

,outcome,x,z,x^2,x^3,x^4
0,17.831014,0.000000,3.000000,0.000000,0.000000,0.000000
1,15.609243,0.102041,3.102041,0.010412,0.001062,0.000108
2,19.957243,0.204082,3.204082,0.041649,0.008500,0.001735
3,17.246179,0.306122,3.306122,0.093711,0.028687,0.008782
4,16.070335,0.408163,3.408163,0.166597,0.067999,0.027755
5,20.877085,0.510204,3.510204,0.260308,0.132810,0.067760
6,23.309690,0.612245,3.612245,0.374844,0.229496,0.140508
7,23.826884,0.714286,3.714286,0.510204,0.364431,0.260308
8,24.235700,0.816327,3.816327,0.666389,0.543991,0.444074
9,24.725722,0.918367,3.918367,0.843399,0.774550,0.711321


In [5]:
linreg.summary()

In [ ]:
linreg.summary(content_type='simple')

In [ ]:
string = '~1'

'~' in string